# 지니차트 Top200 차트

In [147]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


### 1.인터넷에서 데이터 가져오기

In [148]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
req.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [ ]:
# Chrome User-Agent  
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
req = requests.get(url, headers=header)
html = req.text
html

In [150]:
soup = BeautifulSoup(req.text)

### 1.원하는 데이터 찾기

In [151]:
trs = soup.select_one('.list-wrap >tbody').find_all('tr')
len(trs)

50

In [152]:
#그밖에 방법
#  1.  trs = soup.select('tr.list')
#  2.  웹창에서 마우스오른쪽 copy> copy selecter
#      body-content > div.newest-list > div > table > tbody > tr:nth-child(1)

### 2.하나를 선택해서 원하는 정보 추출 해보기

- 순위, 제목, 가수, 앨범명

In [163]:
tr = trs[12]

In [155]:
#순위
ranking = tr.select_one('.number').get_text().split('\n')[0]
ranking

'1'

In [156]:
# #정수로 바꿔줄때 식//예외처리 사용해서 오류를 방지
# try:
#     ranking = int(tr.select_one('.number').get_text().split('\n')[0])
# except:
#     ranking = 0

In [175]:
# 제목
name = tr.select_one('.info> .title.ellipsis').get_text().split('\n')[-1].strip()
name

'어른'

In [158]:
# 가수
singer = tr.select_one('.info> .artist.ellipsis').get_text().strip()
                                    #get_text(), string(), text() 모두 가능
singer

'BIGBANG (빅뱅)'

In [159]:
#앨범명
album = tr.select_one('.info> .albumtitle.ellipsis').get_text().strip()
album

'봄여름가을겨울 (Still Life)'

### 3.반복문 사용해서 50개 모두 가져오기

In [166]:
line = []
for tr in trs:
    ranking = tr.select_one('.number').get_text().split('\n')[0]
    name = tr.select_one('.info> .title.ellipsis').get_text().split('\n')[-1].strip()
    singer = tr.select_one('.info> .artist.ellipsis').get_text()
    album = tr.select_one('.info> .albumtitle.ellipsis').get_text()
    line.append({'순위':ranking,'곡명':name, '가수':singer, '앨범명':album})

    

### 4.모든 페이지 가져오기

In [169]:
line = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220415&hh=09&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    trs = soup.select('tr.list')

    for tr in trs:
        ranking = tr.select_one('.number').get_text().split('\n')[0]
        name = tr.select_one('.info> .title.ellipsis').get_text().split('\n')[-1].strip()
        singer = tr.select_one('.info> .artist.ellipsis').get_text()
        album = tr.select_one('.info> .albumtitle.ellipsis').get_text()
        line.append({'순위':ranking,'곡명':name, '가수':singer, '앨범명':album})
        

### 5.데이터프레임으로 만들기

In [170]:
df = pd.DataFrame(line)
df

,순위,곡명,가수,앨범명
0,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2,LOVE DIVE,IVE (아이브),LOVE DIVE
2,3,TOMBOY,(여자)아이들,I NEVER DIE
3,4,Feel My Rhythm,Red Velvet (레드벨벳),'The ReVe Festival 2022 - Feel My Rhythm'
4,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)
...,...,...,...,...
195,196,하루하루,BIGBANG (빅뱅),Stand Up (2008 빅뱅 3rd Mini Album)
196,197,붉은 노을,BIGBANG (빅뱅),REMEMBER
197,198,Saturday Drip,NCT DREAM,Glitch Mode - The 2nd Album
198,199,"너, 너",스트레이 (The Stray),her


### 6.저장하기

In [171]:
df.to_csv('지니차트 Top 200.csv',index=False)